# Bitcoin price prediction using LSTM


In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from datetime import datetime
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam

from helper import get_fng_index, split_data


## Read Data

In [ ]:
df = pd.read_csv("data/kucoin_btc_usdt1d.csv")

df = df.set_index(pd.DatetimeIndex(df.iloc[:,0].values)).iloc[:,1:]

df

In [ ]:
fng_df = get_fng_index()
fng_df

In [ ]:
data = pd.merge(df, fng_df, on='timestamp')
data

## Data Preparation

### Normalization

In [ ]:
selected_data = data[['close','volume','high','low','open','value_classification']]
num_features = selected_data.shape[1]

In [ ]:
scaler = MinMaxScaler()
norm_data = scaler.fit_transform(selected_data.values)
print("Real: {}\nNormalized: {}".format(selected_data.values[0],norm_data[0]))

### Data split

In [ ]:
x_train, x_test, y_train, y_test = split_data(norm_data,10,1,80)


## Build the model

In [ ]:
num_units = 100
activation_function = 'relu'
loss_function = 'mean_absolute_error'
batch_size = 32
num_epochs = 20

model = Sequential()
model.add(LSTM(units = num_units,return_sequences=True, input_shape=(None, num_features)))

model.add(LSTM(units=num_units))
model.add(Dropout(0.2))

model.add(Dense(units = 1))
model.add(Activation(activation_function))
model.compile(optimizer=Adam(learning_rate=0.0001), loss=loss_function)
model.summary()

## Train the model

In [ ]:
t1 = datetime.now()
kf = KFold(n_splits = 10, shuffle = True)
loss = []
for train_index, test_index in kf.split(x_train):
    x_trn, x_tst = x_train[train_index], x_train[test_index]
    y_trn, y_tst = y_train[train_index], y_train[test_index]
    history = model.fit(
        x_trn,
        y_trn,
        batch_size=32,
        epochs=20,
        shuffle=False)
    
    loss.extend(history.history['loss'])
t2 = datetime.now()
print(t2-t1)

In [ ]:
plt.figure()

plt.plot(range(len(loss)), loss, 'blue', label='Training loss')

plt.title("Training Loss")
plt.xlabel("Epoch")
plt.legend()

plt.show()


In [ ]:
original_train = y_train - scaler.min_[0]
original_train /= scaler.scale_[0]
original_train = pd.DataFrame((original_train))

predictions_train = model.predict(x_train) - scaler.min_[0]
predictions_train /= scaler.scale_[0]
predictions_train = pd.DataFrame((predictions_train))

plt.figure()

plt.plot(original_train, 'blue', label='Train Data')
plt.plot(predictions_train, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()
print("Train error:")
print(mean_absolute_error(y_train, model.predict(x_train)))

## Prediction

In [ ]:
original_test = y_test - scaler.min_[0]
original_test /= scaler.scale_[0]
original_test = pd.DataFrame((original_test))

predictions_test = model.predict(x_test) - scaler.min_[0]
predictions_test /= scaler.scale_[0]
predictions_test = pd.DataFrame((predictions_test))

plt.figure()

plt.plot(original_test, 'blue', label='Test Data')
plt.plot(predictions_test, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()
print("Test error:")
print(mean_absolute_error(y_train, model.predict(x_train)))
print(mean_absolute_error(y_test[:-1], model.predict(x_test[:-1])))

In [ ]:
prediction = pd.DataFrame({"actual price":original_test[0],"predicted":predictions_test[0]})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prediction)


In [ ]:
model.save('saved_models/lstm')